# Trace Collection: Different devices, different executables

---
We will be collecting traces from different devices. Each device will have mutliple traces fro different executables, such as ANSII+masked, MBEDTLS, HWAES

---

In [ ]:
#SCOPETYPE = 'OPENADC'
#PLATFORM = 'CW308_STM32L4' #'CWLITEARM'
#PLATFORM = 'CW308_STM32F4'
PLATFORM = 'CW308_STM32F3'

# #CRYPTO_TARGET = "HWAES" #"MBEDTLS"
# CRYPTO_TARGET = None
SS_VER = 'SS_VER_1_1'
# #CHECK_CORR = False ##From CW_lite code

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.default_setup()
target = cw.target(scope)
# target.baud = 20834
print(target.baud)
print(scope)

In [ ]:
#fw_path = '../../../hardware/victims/firmware/simpleserial-aes-checkTraces/simpleserial-aes-{}.hex'.format(PLATFORM)
#fw_path = '/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32L4_MASKEDAES_ANSSI.hex'
#fw_path = '/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32L4_TINYAES128C.hex'
#fw_path = '/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32L4_MASKEDAES_ANSSI+UNROLLED.hex'
#fw_path = '/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32L4_MBEDTLS.hex'
#fw_path = '/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32L4_HWAES.hex'

#fw_path = '/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32F3_TINYAES128C.hex'
fw_path = '/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32F3_MBEDTLS.hex'
#fw_path = "/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32F4_TINYAES128C.hex"

#fw_path  = "/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32F4_MBEDTLS.hex"

cw.program_target(scope, prog, fw_path, baud=38400)

In [ ]:
scope.default_setup()

In [ ]:
print(scope.adc.presamples)
print(scope.adc.timeout)
print(scope.clock.adc_src)
print(scope.clock.adc_freq)
print(scope.clock.adc_rate)
print(scope.clock.adc_locked)
print(scope.clock.freq_ctr)
print(scope.clock.freq_ctr_src)
print(scope.clock.clkgen_src)
print(scope.clock.extclk_freq)
print(scope.clock.clkgen_mul)
print(scope.clock.clkgen_div)
print(scope.clock.clkgen_freq)
print(scope.gain)

In [ ]:
#scope.adc.samples = 1500
#scope.adc.offset = 24400
#scope.clock.clkgen_freq = 4e6
scope.clock.adc_src = "clkgen_x1"
#scope.gain.db = 30
# target = cw.target(scope)
#target.baud = 20834
print(target.baud)
#scope.adc.timeout = 60
#scope.adc.default()

In [ ]:
reset_target(scope)

In [ ]:
##For 1500 CW downsamples, trace_key_1500
#scope.adc.decimate = 4
scope.adc.samples = 1500 ##Number of samples to take
scope.adc.presamples = 0

In [ ]:
cw.list_devices()

In [ ]:
import json
def saveParams(filePath):
    config={} 
    config["device_name"] = cw.list_devices()[0]["name"]
    config["serial_num"] = cw.list_devices()[0]["sn"]
    config["hw_loc"] = cw.list_devices()[0]["hw_loc"]
    config["adc_presamples"] = scope.adc.presamples
    config["adc_samples"] = scope.adc.samples
    config["adc_timeout"] = scope.adc.timeout
    config["clock_adc_src"] = scope.clock.adc_src
    config["clock_adc_freq"] = scope.clock.adc_freq
    config["scope.clock.adc_rate"] = scope.clock.adc_rate
    config["scope.clock.adc_locked"] = scope.clock.adc_locked
    config["scope.clock.freq_ctr"] = scope.clock.freq_ctr
    config["scope.clock.freq_ctr_src"] = scope.clock.freq_ctr_src
    config["scope.clock.clkgen_src"] = scope.clock.clkgen_src
    config["scope.clock.extclk_freq"] = scope.clock.extclk_freq
    config["scope.clock.clkgen_mul"] = scope.clock.clkgen_mul
    config["scope.clock.clkgen_div"] = scope.clock.clkgen_div
    config["scope.clock.clkgen_freq"] = scope.clock.clkgen_freq
    config["scope_gain_mode"] = scope.gain.mode
    config["scope_gain_gain"] = scope.gain.gain
    config["scope_gain_db"] = scope.gain.db
    
    print(config)
    
    with open(filePath, 'w') as f:
        json.dump(config, f, indent=4)
    print(f"Config file saved to {filePath}")
    return config

In [ ]:
print(scope.adc.presamples)
print(scope.adc.samples)
print(scope.adc.timeout)
print(scope.clock.adc_src)
print(scope.clock.adc_freq)
print(scope.clock.adc_rate)
print(scope.clock.adc_locked)
print(scope.clock.freq_ctr)
print(scope.clock.freq_ctr_src)
print(scope.clock.clkgen_src)
print(scope.clock.extclk_freq)
print(scope.clock.clkgen_mul)
print(scope.clock.clkgen_div)
print(scope.clock.clkgen_freq)
print(scope.gain)

In [ ]:
filePath = "/mnt/hgfs/trace_key_f303_1500_20kAdd_082624/config.json"
config = saveParams(filePath)

In [ ]:
# config

In [ ]:
# with open(filePath, 'w') as f:
#         json.dump(config, f, indent=4)

In [ ]:
#Capture Traces
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()
ktp.fixed_key = False

traces = []
N = 20000  # Number of traces
prjDir = "/mnt/hgfs/trace_key_f303_1500_20kAdd_082624/"

for fileNum in range(256):
    project = cw.create_project(prjDir + "key_" + str(fileNum) + ".cwp", overwrite=True)

    for i in trange(N, desc='Capturing traces:' + str(fileNum)):
        key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
        key[0] = fileNum

        trace = cw.capture_trace(scope, target, text, key)
        if trace is None:
            continue
        project.traces.append(trace)

    try:
        print(scope.adc.trig_count) # print if this exists
    except:
        pass
    project.save()
    project.close()

In [ ]:
np.random.choice(256,1)[0]

In [ ]:
#Capture Traces
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()
ktp.fixed_key = False

traces = []
N = 10  # Number of traces
project = cw.create_project("traces/f303_trial_082624_cwlite.cwp", overwrite=True)

for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    key[0] = 0
#     for k in range(len(key)):
#         key[k] = 150
        
    #print(f"key= {key}")

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

try:
    print(scope.adc.trig_count) # print if this exists
except:
    pass
project.save()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import chipwhisperer as cw

project = cw.open_project("traces/f303_trial.cwp")
print(project.location)
xShape = project.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)

#savePath = "/home/manojgopale/Documents/chipwhisperer/jupyter/courses/sca201/img/sca201_images/" + prjName + ".jpg"

fig, axs = plt.subplots(4, sharex=True, sharey= True, figsize=(30,10))
axs[0].plot(x,project.traces[0].wave)
axs[0].set_title(project.traces[0].key)
axs[1].plot(x,project.traces[1].wave)
axs[1].set_title(project.traces[1].key)
axs[2].plot(x,project.traces[8].wave)
axs[2].set_title(project.traces[8].key)
axs[3].plot(x,project.traces[9].wave)
axs[3].set_title(project.traces[9].key)

fig.savefig("img/l443_1500.png")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import chipwhisperer as cw
import plotly.graph_objects as go
from plotly.subplots import make_subplots

project = cw.open_project("traces/f303_trial_082624_cwlite.cwp")

print(project.location)
xShape = project.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=1, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
for idx in range(4):
    fig.add_trace(go.Scatter(x=x, y=project.traces[idx].wave, mode='lines', line=dict(color=colorList[idx]), name="trace_" + str(idx)))

fig.update_layout(
    showlegend=True,
    title="F303 traces, hex from github"
)

savePath = "img/f303_trial_082024_cwlite.html"
fig.write_html(savePath, auto_open=True)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

project1 = cw.open_project("traces/ansii_masked_unrolled_7p38mhzclock_clk_x1.cwp")
project2 = cw.open_project("traces/ansii_masked_unrolled_7p38mhzclock_clk_x1_24400_offset.cwp")
print(project.location)
traceList=[]
keyList = []
df = pd.DataFrame(columns=['trace', 'key'])

for idx, (trace1, trace2) in enumerate(zip(project1.traces,project2.traces)):
    #print(f"trace1= {trace1.wave.shape}, trace2= {trace2.wave.shape}")
    traceList.append(np.concatenate((trace1.wave,trace2.wave), axis=0))
    keyList.append(trace1.key)

df["trace"] = traceList
df["key"] = keyList

#print(f"fullData shape= {len(traceList)}\ndf= {df.head}\ndf_1 = {df.iloc[0].shape}")

xShape = df.trace.iloc[0].shape[0]
x=np.linspace(0,xShape-1, xShape)

#savePath = "/home/manojgopale/Documents/chipwhisperer/jupyter/courses/sca201/img/sca201_images/" + prjName + ".jpg"

fig, axs = plt.subplots(4, sharex=True, figsize=(30,10))
axs[0].plot(x,df.trace.iloc[0])
axs[0].set_title(df.key.iloc[0])
axs[1].plot(x,df.trace.iloc[1])
axs[1].set_title(df.key.iloc[1])
axs[2].plot(x,df.trace.iloc[8])
axs[2].set_title(df.key.iloc[8])
axs[3].plot(x,df.trace.iloc[9])
axs[3].set_title(df.key.iloc[9])

fig.savefig("img/ansii_masked_unrolled_7p38mhzclock_clk_x1_48800TimeSamples.png")

In [1]:
## Use this to save to npz, pkl format is not compatible across different versions of python.
from tqdm import tnrange, trange
import chipwhisperer as cw
import pandas as pd
import os
import numpy as np

## dataFrame to store all the traces
df_new = pd.DataFrame(columns=['trace', 'key'])
traceList = []
keyList = []

trainSize= 20000
devSize = 500
testSize = 500
# ##Adding traces for not complete 
# keysNotDone = [204, 228, 244, 245]
# count = 0
prjDir = "/mnt/hgfs/trace_key_f303_1500_20kAdd_082624/"
saveDir = "/mnt/hgfs/trace_key_f303_1500_20kAdd_082624_dataset/"

for fileNum in trange(256, desc='Splitting data'):
    df_new = pd.DataFrame(columns=['trace', 'key'])
    traceList = []
    keyList = []
    print("Started key= %s" %(fileNum))
    prjName = "key_" + str(fileNum)
    new_project = cw.open_project(prjDir + prjName + ".cwp")
    for index, trace in enumerate(new_project.traces):
        #df_new = df_new.append({'trace': trace.wave, 'key': trace.key}, ignore_index=True)
        traceList.append(trace.wave)
        keyList.append(trace.key)

    ##Create trace and key dataFrame        
    df_new["trace"] = traceList
    df_new["key"] = keyList

    ##Shuffle the dataframe before splitting in test, dev and train
    df_new = df_new.sample(frac=1).reset_index(drop=True)
    
    ##Split it in train, dev and test
    df_train = df_new.iloc[0:trainSize, : ].reset_index(drop=True)
# #     df_dev   = df_new.iloc[trainSize:trainSize+devSize, : ].reset_index(drop=True)
# #     df_test  = df_new.iloc[trainSize+devSize:trainSize+devSize+testSize, : ].reset_index(drop=True)
    
    ##Save it to pkl
    saveTrain = saveDir + "/train_" + str(fileNum) + ".npz"
# #     saveDev   = saveDir + "/dev_" + str(fileNum) + ".npz"
# #     saveTest  = saveDir + "/test_" + str(fileNum) + ".npz"

    if (not os.path.exists(saveTrain)):
        np.savez_compressed(saveTrain, data=df_train)
        print("Saved fileNum= %s to %s" %(fileNum, saveTrain))
        
# #     if (not os.path.exists(saveDev)):
# #         np.savez_compressed(saveDev, data=df_dev)
# #         print("Saved key= %s to %s" %(fileNum, saveDev))
    
# #     if (not os.path.exists(saveTest)):
# #         np.savez_compressed(saveTest, data=df_test)
# #         print("Saved key= %s to %s" %(fileNum, saveTest))

    print("Done saving files for file= %s" %(fileNum))
    ##Clean al dataframe's before next loop
    df_new = df_new.drop(df_new.columns, axis=1)
    df_train = None
#     df_dev = None
#     df_test = None


## Add these to the csv files.
print("Done saving all the traces\n")

Splitting data:   0%|                                                                                                                                                         | 0/256 [00:00<?, ?it/s]

Started key= 0


Splitting data:   0%|▌                                                                                                                                              | 1/256 [00:27<1:55:45, 27.24s/it]

Saved fileNum= 0 to /mnt/hgfs/trace_key_f303_1500_20kAdd_082624_dataset//train_0.npz
Done saving files for file= 0
Started key= 1


Splitting data:   1%|█                                                                                                                                              | 2/256 [00:52<1:51:22, 26.31s/it]

...

Saved fileNum= 253 to /mnt/hgfs/trace_key_f303_1500_20kAdd_082624_dataset//train_253.npz
Done saving files for file= 253
Started key= 254


Splitting data: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 255/256 [1:54:44<00:26, 26.88s/it]

Saved fileNum= 254 to /mnt/hgfs/trace_key_f303_1500_20kAdd_082624_dataset//train_254.npz
Done saving files for file= 254
Started key= 255


Splitting data: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [1:55:10<00:00, 26.99s/it]

Saved fileNum= 255 to /mnt/hgfs/trace_key_f303_1500_20kAdd_082624_dataset//train_255.npz
Done saving files for file= 255
Done saving all the traces



In [ ]:
from tqdm import tnrange, trange
import chipwhisperer as cw
import pandas as pd
import os
import numpy as np

## dataFrame to store all the traces
df_new = pd.DataFrame(columns=['trace', 'key'])
traceList = []
keyList = []

trainSize= 7500
devSize = 500
testSize = 500
# ##Adding traces for not complete 
# keysNotDone = [204, 228, 244, 245]
# count = 0
prjDir = "/mnt/hgfs/trace_key_f303_tinyAES/"
saveDir = "/mnt/hgfs/compressed_dataset/"

fileNume=50
prjName = "file_" + str(fileNum)
new_project = cw.open_project(prjDir + prjName + ".cwp")
trace = new_project.traces

keylist=[]
for idx, traces in enumerate(trace):
    keyList.append(traces.key)
    


In [ ]:
import collections
#ounter = collections.Counter(np.array(keyList)[:,0])
#counter
collection = collections.Counter(df_new.key.apply(lambda x: x[0]))
collection

In [ ]:
scope.dis()
target.dis()

In [ ]:
#ktp.fixed_key =False
ktp.next()